In [ ]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 8.7MB/s 
     |████████████████████████████████| 870kB 15.6MB/s 
     |████████████████████████████████| 1.2MB 34.7MB/s 
     |████████████████████████████████| 5.6MB 43.8MB/s 
     |████████████████████████████████| 133kB 56.2MB/s 
     |████████████████████████████████| 7.4MB 40.8MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=9e4665b6b86ec79662ac447b98e0a665650a08b8cbff1dc09d8cc91eaef66c5a
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
ERROR: botocore 1.20.52 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import torch
import pandas as pd
import numpy as np
from yaml import load, dump, Loader, Dumper
print(torch.__version__)

1.8.1+cu101


In [ ]:
import yaml

with open("nlu.yml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
[example.replace('- ', '') for example in data['nlu'][11]['examples'].split('\n')]

['is the fur used in your products real?',
 'are all ASOS products vegan?',
 'do you use vegan fur?',
 'What kind of fur do you use?',
 'Does your company use real fur?',
 'Is your fur cruelty free?',
 'Are your products made of real animal fur?',
 'Does your product contain real fur?',
 'Is the fur on your product real?',
 'Is your fur fake?',
 'Is your fur real?',
 'Is any of the product on the website made of real fur?',
 'wats your fur made of',
 '']

In [ ]:
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)


device  cuda


In [ ]:
def generate_sentences(sentence):


  text =  "paraphrase: " + sentence + " </s>"
  print(text)


  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.99,
      early_stopping=True,
      num_return_sequences=10
  )
  final_outputs=[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)

  return final_outputs[0:10]

In [ ]:
[example.replace('- ', '') for example in data['nlu'][11]['examples'].split('\n')]

['is the fur used in your products real?',
 'are all ASOS products vegan?',
 'do you use vegan fur?',
 'What kind of fur do you use?',
 'Does your company use real fur?',
 'Is your fur cruelty free?',
 'Are your products made of real animal fur?',
 'Does your product contain real fur?',
 'Is the fur on your product real?',
 'Is your fur fake?',
 'Is your fur real?',
 'Is any of the product on the website made of real fur?',
 'wats your fur made of',
 '']

In [ ]:
augmented_data = {}
for i in range(11, len(data['nlu'])):
  intent = data['nlu'][i]['intent']
  examples = [example.replace('- ', '') for example in data['nlu'][i]['examples'].split('\n')]
  generated_sentences = []
  for example in examples:
    generated_sentences.extend(generate_sentences(example))
  augmented_data[intent] = generated_sentences
  print(f'{np.round(100*(i-10)/39, 2)}% complete')



Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: is the fur used in your products real? </s>


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: are all ASOS products vegan? </s>


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


...

paraphrase: Can I use a different currency on the app? </s>


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: Which currencies can I use on the app? </s>


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: Can I choose a different currency on the app? </s>
100.0% complete


In [ ]:
with open('augmented_data.yml', 'w') as outfile:
    yaml.dump(augmented_data, outfile, default_flow_style=False)